In [1]:
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import plotly.graph_objects as go

In [2]:
if not os.path.exists('persistence_plot'): os.makedirs('persistence_plot')

Dr = 0.005642764926868786
range_value = 899
percent_to_fit = 10
msd_range = 44999 # (nstep-neq)/ 2*nstore - 1

def find_file_paths(parent_folder, target_file):
    file_paths = []

    # Iterate through all directories and subdirectories using os.walk, Check if the target file exists in the current directory, If the file is found, append its path to the list
    for dirpath, _, filenames in os.walk(parent_folder):
        if target_file in filenames: file_paths.append(os.path.join(dirpath, target_file))
    file_paths.sort()
    return file_paths

parentFolder = r"D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\pf_0.0785"
targetFile = "Persistence.dat"
# targetFile = "msd1.dat"
filePaths = find_file_paths(parentFolder, targetFile)

# Extract the packing fraction from the parent folder path
packing_fraction = parentFolder.split('/')[-1].split('_')[-1]


def extract_numbers(file_paths):
    numbers_list = []  # Initialize an empty list to store the extracted numbers

    # Iterate through each path in the file_paths list
    for path in file_paths:
        # Split the path by '/' to extract the desired part
        parts = path.replace('\\', '/').split('/')

        # Iterate through the parts to find the number
        for part in parts:
            # Check if the part starts with 'pf_' followed by a number
            if part.startswith('pf_') and part.count('_') == 2:
                # Extract the number between '_' and '/'
                number = part.split('_')[-1]
                numbers_list.append(int(number))
                break  # Break the loop once the number is found

    return numbers_list

theta_list = extract_numbers(filePaths)

In [3]:
def extract_dat_from_file(file_path):
    t_values = []
    a_values = []
    p_values = []

    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            # Assuming values are separated by whitespace
            values = line.split()
            t_values.append(float(values[0]))  # Assuming x values are in the first column
            a_values.append(float(values[1]))  # Assuming y values are in the second column
            p_values.append(float(values[2]))  # Assuming z values are in the third column
            if i > range_value: break
    return t_values, a_values, p_values

def extract_data_for_msd(file_path):
    t_values, r2_values = [], []
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            values = line.split()
            t_values.append(float(values[0]))  # Assuming x values are in the first column
            r2_values.append(float(values[3]))  # Assuming y values are in the second column
            if i >= msd_range: break
    return t_values, r2_values

In [4]:
# Function to fit
def Persistence_fit(x, A, b, c):
    return A * np.exp(-((np.array(x) / b)**c))

def linear_fit(x, b, c): return 4 * b * np.array(x) + c
# def Correlation_fit(x, A, omega, c, k, Beta):
#     return A * (np.cos(2*math.pi*np.array(x)*omega)+c) * np.exp(-(np.array(x)/k)**Beta)


# MSD

In [ ]:
def plot_MSD():
    allowed= [10, 20, 55, 65, 80, 90]
    for j, file_path in enumerate(filePaths):
        theta = theta_list[j]
        if theta not in allowed: continue
        x, y = extract_data_for_msd(file_path)
        x = [i * Dr for i in x]
        # define xfit as the last 10% of the x values
        start_index = int(len(x) * (1 - percent_to_fit / 100))
        xfit = x[start_index:]
        yfit = y[start_index:]
        params, _ = curve_fit(linear_fit, xfit, yfit)
        print(f"Parameters for curve {theta}: b = {params[0]:.6f}")

        # Plot the fitted curve and points with the same color
        color = plt.cm.tab20(theta / 90)  # Using colormap for different colors
        label_fit = f'{theta}$^\\circ$'  # Include c parameter in the legend
        plt.plot(x, y, '*', color=color, label=label_fit, markersize=5)

    plt.tick_params(axis='both', which='major', direction='in', length=8, width=2, bottom=True, top=True, left=True, right=True)
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=2, bottom=True, top=True, left=True, right=True)

    # plt.minorticks_on()
    plt.xscale("log")
    plt.yscale("log")
    plt.xlabel('$D_{r}t$', fontsize=20)
    plt.ylabel('$\\langle r^{2} ( t ) \\rangle/ \\sigma^{2} $', fontsize=20)
    plt.legend(loc='upper left', fontsize=15, markerscale=2)
    plt.title(f'$\\phi$ = {packing_fraction}', fontsize=20)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    fig = plt.gcf()
    fig.set_size_inches(10, 5)
    plt.savefig(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\MSD_Plot\MSD_Fits.png', dpi=600, bbox_inches='tight', facecolor='w', edgecolor='b')
plot_MSD()

In [ ]:
def plot_MSD_fit():
    names = []
    fit_params = []
    for j, file_path in enumerate(filePaths):
        theta = theta_list[j]
        names.append(theta)
        x, y = extract_data_for_msd(file_path)
        x = [i * Dr for i in x]
        start_index = int(len(x) * (1 - percent_to_fit / 100))
        xfit = x[start_index:]
        yfit = y[start_index:]
        params, _ = curve_fit(linear_fit, xfit, yfit)
        fit_params.append(params[0])
        print(f"Parameters for curve {theta}: b = {params[0]:.6f}")

    plt.figure(figsize=(10, 5))
    plt.plot(names, fit_params, color='red', marker='o', linestyle='--', linewidth=2)
    plt.tick_params(axis='both', which='major', direction='in', length=8, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.minorticks_on()
    plt.xlabel('$\\theta^\\circ$', fontsize=20)
    plt.ylabel('$D_t$', fontsize=20)
    plt.title(f'$\\phi$ = {packing_fraction}', fontsize=20)
    plt.savefig('msd_plot/Fit_Parameter_Df_vs_Vision_Cone.png', dpi=600, bbox_inches='tight')

    data = np.column_stack((names, fit_params))
    np.savetxt(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\MSD_Plot\fit_parameters_MSD.txt', data, fmt='%.6f', header='MSD Fit_Parameters', delimiter='\t')
plot_MSD_fit()

# Persistence

In [ ]:
def plot_persistence():
    allowed= [10, 20, 55, 65, 80, 90]
    for j,file_path in enumerate(filePaths):
        theta = theta_list[j]
        if theta not in allowed: continue

        x, _, z = extract_dat_from_file(file_path)
        x = [i * Dr for i in x]        # multiply by Dr to get the actual time
        params, _ = curve_fit(Persistence_fit, x[5:], z[5:], p0=[0.5, 0.5, 0.5])
        print(f"Parameters for curve {theta}: A = {params[0]:.10f}, k = {params[1]:.10f}, Beta = {params[2]:.10f}")
        color = plt.cm.tab20(theta / 90)  # Using colormap for different colors
        label_fit = f'{theta}$^\\circ$'  # Include c parameter in the legend/
        plt.plot(x, Persistence_fit(x, *params), '--', color=color)
        plt.plot(x, z, '*', color=color, label=label_fit, markersize=5)

    plt.tick_params(axis='both', which='major', direction='in', length=8, width=2, bottom=True, top=True, left=True, right=True)
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=2, bottom=True, top=True, left=True, right=True)
    plt.minorticks_on()
    plt.xscale("log")
    plt.xlim(0.01, None)
    plt.ylim(0, 1.0)
    plt.xlabel('$D_{r}t$', fontsize=20,) 
    plt.ylabel('$\\zeta(t)$', fontsize=20)
    plt.title(f'$\\phi$ = {packing_fraction}', fontsize=20)
    #increase the fontsize of the x and y values
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

    # Rotate x-axis labels to prevent overlapping
    # plt.xticks(rotation=45)

    plt.legend(loc='upper right', fontsize=15, markerscale=2)
    fig = plt.gcf()
    fig.set_size_inches(10, 5)
    plt.savefig(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\persistence_plot\Persistence.png', dpi=600, bbox_inches='tight', facecolor='w', edgecolor='b')
    plt.close()
plot_persistence()

In [6]:
def plot_persistence():
    fig = go.Figure()
    for j,file_path in enumerate(filePaths):
        theta = theta_list[j]

        x, y, z = extract_dat_from_file(file_path)
        x = [i * Dr for i in x]
        params, covariance = curve_fit(Persistence_fit, x[:], z[:], maxfev=20000)
        params, covariance = curve_fit(Persistence_fit, x[:], z[:], maxfev=20000, method='trf')
        print(f"Parameters for curve {theta}: a = {params[0]:.10f}, b = {params[1]:.10f}, c = {params[2]:.10f}")
        color = plt.cm.tab20(theta / 90)
        fig.add_trace(go.Scatter(x=x, y=z, mode='markers', name=f'{theta}° Data', marker=dict(color=f'rgba{color}')))
        fig.add_trace(go.Scatter(x=x, y=Persistence_fit(x, *params), mode='lines', name=f'{theta}° Fit', line=dict(color=f'rgba{color}')))

    fig.update_layout(
        xaxis_type="log",
        xaxis_title='$D_{r}t$',
        yaxis_title='$\\zeta(\\Delta t)$',
        title=f'Packing fraction = {packing_fraction}',
        width=1500,
        height=900,
    )
    fig.update_xaxes(range=[-2, None])
    fig.write_html(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\persistence_plot\Persistence.html')
    fig.show()
plot_persistence()

Parameters for curve 10: a = 1.0030167358, b = 0.4778173312, c = 1.1286682944
Parameters for curve 15: a = 0.9897920022, b = 0.4227659320, c = 1.0983836046
Parameters for curve 20: a = 0.9824594503, b = 0.3774222008, c = 1.0283583417
Parameters for curve 25: a = 0.9868499662, b = 0.1255199558, c = 0.5465685222
Parameters for curve 30: a = 0.9966864212, b = 0.0960479646, c = 0.5038338316
Parameters for curve 35: a = 0.9970031575, b = 0.0806869571, c = 0.4766519981
Parameters for curve 40: a = 0.9807301853, b = 0.0844227091, c = 0.4762636673
Parameters for curve 45: a = 0.9381232580, b = 0.1123874482, c = 0.4994534078


C:\Users\alanb\AppData\Local\Temp\ipykernel_21252\2581450875.py:3: RuntimeWarning:

invalid value encountered in power



Parameters for curve 50: a = 0.9262819338, b = 0.1004655476, c = 0.4632923241
Parameters for curve 55: a = 0.7715709492, b = 0.2672165609, c = 0.6820782826
Parameters for curve 60: a = 0.8418802807, b = 0.8229772103, c = 1.6365183682
Parameters for curve 65: a = 0.5895098704, b = 0.5545947505, c = 1.2499248669
Parameters for curve 70: a = 0.9435315048, b = 0.0706505143, c = 0.4146590675
Parameters for curve 75: a = 1.0133824859, b = 0.0146179457, c = 0.4267696872
Parameters for curve 80: a = 1.0055494472, b = 0.0136062265, c = 0.5572949652
Parameters for curve 85: a = 1.0031305557, b = 0.0119224563, c = 0.6146201967
Parameters for curve 90: a = 1.0028090147, b = 0.0119847136, c = 0.6293893106


C:\Users\alanb\AppData\Local\Temp\ipykernel_21252\2581450875.py:3: RuntimeWarning:

divide by zero encountered in power



In [ ]:
def plot_persistence_fit():
    names = []
    fit_params_b = []
    fit_params_inv_b = []
    fit_params_c = []
    for j,file_path in enumerate(filePaths):
        theta = theta_list[j]
        names.append(theta)
        x, _, z = extract_dat_from_file(file_path)
        x = [i * Dr for i in x]
        params, _ = curve_fit(Persistence_fit, x, z, p0=[0.5, 0.5, 0.5])
        fit_params_b.append(params[1])
        fit_params_inv_b.append(1 / params[1])
        fit_params_c.append(params[2])
        print(f"Parameters for curve {theta}: a = {params[0]:.10f}, b = {params[1]:.10f}, 1/b = {1 / params[1]:.10f}, c = {params[2]:.10f}")

    #plotting beta vs angle for persistence

    plt.figure(figsize=(10, 5))
    plt.plot(names, fit_params_c, color='red', marker='o', linestyle='--', linewidth=2)
    # Customize tick marks for both major and minor ticks
    plt.tick_params(axis='both', which='major', direction='in', length=8, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.minorticks_on()
    plt.xlabel(' $\\theta^\\circ$', fontsize=20)
    plt.ylabel(f'$\\beta$', fontsize=20)
    plt.title(f'$\\phi$ = {packing_fraction}', fontsize=20)
    plt.savefig(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\persistence_plot\persistence_beta.png', dpi=600, bbox_inches='tight')
    plt.close()
    #plotting tau vs angle for persistence

    plt.figure(figsize=(10, 5))
    plt.plot(names, fit_params_inv_b, color='red', marker='o', linestyle='--', linewidth=2)
    # Customize tick marks for both major and minor ticks
    plt.tick_params(axis='both', which='major', direction='in', length=8, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.minorticks_on()
    plt.xlabel('$\\theta^\\circ$', fontsize=20)
    plt.ylabel(f'$\\tau$', fontsize=20)
    plt.title(f'$\\phi$ = {packing_fraction}', fontsize=20)
    plt.savefig(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\persistence_plot\persistence_tau.png', dpi=600, bbox_inches='tight') 

    # Save b, 1/b, and c values
    data = np.column_stack((names, fit_params_b, fit_params_inv_b, fit_params_c))
    np.savetxt('persistence_plot/fit_parameters_persistence.txt', data, fmt='%.10f', header='Theta\t\t 1/tau \t\t tau \t\t beta \t', delimiter='\t')

    plt.close()
plot_persistence_fit()


# Auto-Corelation

In [ ]:
import warnings
def plot_auto_correlation():
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    fig = go.Figure()
    initial_guess = [0.5, 0.5, 0.5, 0.5, 0.5]
    bounds = ([0.0, 0.0, -np.inf, -2, -2], [1.0, 2.0, 2.0, np.inf, 2.0]) # Example bounds, adjust as needed A, omega, c, k, Beta

    for j, file_path in enumerate(filePaths):
        theta = theta_list[j]

        x, y, _ = extract_dat_from_file(file_path)
        x = [i * Dr for i in x]
        label_fit = f'{theta}'
        color = plt.cm.tab20(theta / 90)
        fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name=f'{theta} data', marker=dict(color=f'rgba{color}')))
        if theta<35:Al=abs(max(y[2:])) #comment out the if else for lowdensity
        else:Al=abs(max(y[3:]))
        def Correlation_fit(x, A, omega, c, k, Beta): return Al * (np.cos(2*math.pi*np.array(x)*omega)+c) * np.exp(-(np.array(x)/k)**Beta)/(1+c)# added 1/(1+c) to normalize the data
        try:
            params, _ = curve_fit(Correlation_fit, x[:], y[:], maxfev=20000, method="trf" ,p0=initial_guess, bounds=bounds)
            params[0]=Al
            print(f"Parameters for curve {theta}: A = {params[0]}, omega = {params[1]:.10f}, c = {params[2]:.10f}, k = {params[3]:.10f}, Beta = {params[4]:.10f}")
        except RuntimeError as e:
            print(f"RuntimeError for {theta}: {e}")
            continue  # Skip this iteration if curve fitting fails

        fig.add_trace(go.Scatter(x=x, y=Correlation_fit(x, *params), mode='lines', name=label_fit, line=dict(color=f'rgba{color}')))

    fig.update_layout(
        xaxis_type="log",
        xaxis_title='$D_{r}t$',
        yaxis_title='$\\phi_n(t)$',
        title=f'Packing fraction = {packing_fraction}',
        width=1500,
        height=900,
    )
    fig.update_xaxes(range=[-2, None])
    fig.update_yaxes(range=[-1.0, 1.0])
    fig.write_html(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\AutoC_plot\AutoCorrelation.html')
    fig.show()
plot_auto_correlation()

In [ ]:
import warnings
def plot_auto_correlation():
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    allowed= [10, 20, 55, 65, 80, 90]
    initial_guess = [0.5, 0.5, 0.5, 0.5, 0.5]
    bounds = ([0.0, 0.0, -np.inf, -2, -2], [1.0, 2.0, 2, np.inf, 2]) # Example bounds, adjust as needed A, omega, c, k, Beta
    for j,file_path in enumerate(filePaths):
        theta = theta_list[j]
        if theta not in allowed: continue

        x, y, _ = extract_dat_from_file(file_path)
        x = [i * Dr for i in x]
        color = plt.cm.tab20(theta / 90)

        plt.plot(x, y, 'o', color=color, label=f'{theta}°', markersize=5)
        # if theta<35:Al=abs(max(y[2:])) #comment out the if else for lowdensity
        # else:Al=abs(max(y[3:]))
        Al=abs(max(y[2:]))
        def Correlation_fit(x, A, omega, c, k, Beta): return Al * (np.cos(2*math.pi*np.array(x)*omega)+c) * np.exp(-(np.array(x)/k)**Beta)/(1+c)
        params, _ = curve_fit(Correlation_fit, x[:], y[:], maxfev=20000, method="trf" ,p0=initial_guess, bounds=bounds)
        params[0]=Al
        print(f"Parameters for curve {theta}: A = {params[0]}, omega = {params[1]:.10f}, c = {params[2]:.10f}, k = {params[3]:.10f}, Beta = {params[4]:.10f}")
        plt.plot(x, Correlation_fit(x, *params), '-', color=color)


    plt.tick_params(axis='both', which='major', direction='in', length=8, width=2, bottom=True, top=True, left=True, right=True)
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=2, bottom=True, top=True, left=True, right=True)
    plt.minorticks_on()
    plt.xscale("log")
    plt.xlim(0.01, None)
    plt.ylim(None, 1.0)
    plt.xlabel('$D_{r}t$', fontsize=20)
    plt.ylabel('$\\phi_n(t)$', fontsize=20)
    plt.title(f'$\\phi$ = {packing_fraction}', fontsize=20)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)


    plt.legend(loc='upper right', fontsize=15)
    fig = plt.gcf()
    fig.set_size_inches(10, 5)
    plt.savefig(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\AutoC_plot\AutoCorrelation.png', dpi=600, bbox_inches='tight', facecolor='w', edgecolor='b')
    plt.close()
plot_auto_correlation()

In [ ]:
def plot_auto_correlation_fit():
    names, fit_params_b, fit_params_inv_b, fit_params_c, fit_params_d, metric = [], [], [], [], [], []
    initial_guess = [0.5, 0.5, 0.5, 0.5, 0.5]
    bounds = ([0.0, 0.0, -np.inf, -2, -2], [1.0, 2.0, 2, np.inf, 2]) # Example bounds, adjust as needed A, omega, c, k, Beta

    for j, file_path in enumerate(filePaths):
        theta = theta_list[j]
        names.append(theta)
        x, y, _ = extract_dat_from_file(file_path)
        x = [i * Dr for i in x]
        if theta<35:Al=abs(max(y[2:])) #comment out the if else for lowdensity
        else:Al=abs(max(y[3:]))
        # Al=abs(max(y[2:]))
        def Correlation_fit(x, A, omega, c, k, Beta): return Al * (np.cos(2*math.pi*np.array(x)*omega)+c) * np.exp(-(np.array(x)/k)**Beta)/(1+c)

        try:
            params, _ = curve_fit(Correlation_fit, x[:], y[:], maxfev=20000, method="trf" ,p0=initial_guess, bounds=bounds)
            params[0]=Al
            fit_params_b.append(params[1])
            fit_params_inv_b.append(1 /(params[1]))
            fit_params_c.append(params[4])
            # fit_params_d.append(params[3])
            # metric.append((params[0] / (abs(params[2]) + 1)) * ((params[1]) / (params[3] + 1)))
            metric.append((params[0] / (abs(params[2]) + 1)) * (params[1] * (params[3])))
            # metric.append(params[0] * ((params[1]) / (params[3] + 1)))
            print(f"Parameters for curve {theta}: A = {params[0]}, omega = {params[1]:.10f}, c= {params[2]:.10f}, k = {params[3]:.10f}, Beta = {params[4]:.10f}")
        except RuntimeError as e:
            print(f"RuntimeError for {theta}: {e}")
            fit_params_b.append(0)
            fit_params_inv_b.append(0)
            fit_params_c.append(0)
            fit_params_d.append(0)
            metric.append(0)


    plt.figure(figsize=(10, 5))
    plt.tick_params(axis='both', which='major', direction='in', length=8, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.minorticks_on()
    plt.xlabel('$\\theta^\\circ$', fontsize=20)
    plt.title(f'$\\phi$ = {packing_fraction}', fontsize=20)

    # plotting beta vs angle for autocorrelation
    # plt.plot(names, fit_params_c, color='red', marker='o', linestyle='--', linewidth=2)
    # plt.ylabel(f'$\\beta$', fontsize=20)
    # plt.savefig(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\persistence_plot\autocorrelation_beta.png', dpi=600, bbox_inches='tight')

    #plotting k vs angle for autocorrelation
    # plt.plot(names, fit_params_d, color='red', marker='o', linestyle='--', linewidth=2)
    # plt.ylabel(f'$\\tau$', fontsize=12)
    # plt.savefig(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\persistence_plot\autocorrelation_tau.png', dpi=300, bbox_inches='tight')

    #plotting omega vs angle for autocorrelation
    # plt.plot(names, np.abs(fit_params_b), color='red', marker='o', linestyle='--', linewidth=2)
    # plt.plot(names, np.abs(fit_params_inv_b), color='red', marker='o', linestyle='--', linewidth=2)
    plt.plot(names, np.abs(metric), color='red', marker='o', linestyle='--', linewidth=2)
    plt.ylabel('$metric$', fontsize=12)
    plt.savefig(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\AutoC_plot\autocorrelation_metric.png', dpi=600, bbox_inches='tight')

    plt.show()
    plt.close()

    # Save b, c, and d values with precision up to 10 decimal places
    data = np.column_stack((names, metric))
    np.savetxt(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\AutoC_plot\fit_parameters_autocorrelationm.txt', data, fmt='%.10f', header='Theta\t\t metric \t', delimiter='\t')


plot_auto_correlation_fit()

# Merge

In [ ]:
def plot_MSD_fit_merge():
    # Read the first and second column from the file and plot them
    datal = np.loadtxt(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\MSD_Plot\fit_parameters_MSDl.txt', delimiter='\t', skiprows=1)
    datam = np.loadtxt(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\MSD_Plot\fit_parameters_MSDm.txt', delimiter='\t', skiprows=1)
    names = [f"{int(angle)}°"  for angle in datal[:, 0] ]
    fit_paramsl = datal[:, 1]  * Dr
    fit_paramsm = datam[:, 1]  * Dr
    # add legend
    
    plt.figure(figsize=(10, 5))
    plt.plot(names, fit_paramsl, color='red', marker='s', linestyle='--', linewidth=3, markersize=13, label='$\\phi$ = 0.00785')
    plt.plot(names, fit_paramsm, color='black', marker='^', linestyle='--', linewidth=3, markersize=15, label='$\\phi$ = 0.0785')
    plt.xticks(names[::2], rotation=45)  # Show only alternative labels on the x-axis
    plt.tick_params(axis='both', which='major', direction='in', length=8, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.minorticks_on()
    plt.xlabel('$\\theta$', fontsize=20)
    plt.ylabel('$D_{eff}$', fontsize=20)
    # add legend
    plt.legend(['$\\phi$ = 0.00785', '$\\phi$ = 0.0785'], loc='upper right', fontsize=15)
    
    # plt.show()
    plt.savefig('msd_plot/Fit_Parameter_Df_vs_Vision_Cone.png', dpi=600, bbox_inches='tight')
plot_MSD_fit_merge()

In [ ]:
def plot_persistence_fit_merge():
    # Read the first and second column from the file and plot them
    datal = np.loadtxt(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\persistence_plot\fit_parameters_persistencel.txt', delimiter='\t', skiprows=1)
    datam = np.loadtxt(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\persistence_plot\fit_parameters_persistencem.txt', delimiter='\t', skiprows=1)
    names = datal[:, 0]
    names = [f"{int(angle)}°"  for angle in datal[:, 0] ]
    fit_taul = datal[:, 2]
    fit_taum = datam[:, 2]
    fit_betal = datal[:, 3]
    fit_betam = datam[:, 3]
    # add legend
    plt.figure(figsize=(10, 5))
    plt.plot(names, fit_taul, color='#a2de2e', marker='s', linestyle='--', linewidth=3, markersize=12, label='$\\phi$ = 0.00785')
    plt.plot(names, fit_taum, color='#ae7ff8', marker='o', linestyle='--', linewidth=3, markersize=15, label='$\\phi$ = 0.0785')
    plt.tick_params(axis='both', which='major', direction='in', length=8, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.minorticks_on()
    plt.xticks(names[::2], rotation=45)  # Show only alternative labels on the x-axis
    plt.xlabel('$\\theta^\\circ$', fontsize=20)
    plt.ylabel('$\\tau$', fontsize=20)
    # add legend
    plt.legend(['$\\phi$ = 0.00785', '$\\phi$ = 0.0785'], loc='upper left', fontsize=15)
    plt.savefig('persistence_plot/Fit_Parameter_tau_vs_Vision_Cone.png', dpi=600, bbox_inches='tight')

    plt.figure(figsize=(10, 5))
    plt.plot(names, fit_betal, color='#ef6629', marker='s', linestyle='--', linewidth=3, markersize=12, label='$\\phi$ = 0.00785')
    plt.plot(names, fit_betam, color='#4354f1', marker='o', linestyle='--', linewidth=3, markersize=15, label='$\\phi$ = 0.0785')
    plt.tick_params(axis='both', which='major', direction='in', length=8, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.minorticks_on()
    plt.xticks(names[::2], rotation=45)  # Show only alternative labels on the x-axis
    plt.xlabel('$\\theta$', fontsize=20)
    plt.ylabel('$\\beta$', fontsize=20)
    # add legend
    plt.legend(['$\\phi$ = 0.00785', '$\\phi$ = 0.0785'], loc='upper right', fontsize=15)
    plt.savefig('persistence_plot/Fit_Parameter_Beta_vs_Vision_Cone.png', dpi=600, bbox_inches='tight')

plot_persistence_fit_merge()

In [ ]:
def plot_auto_correlation_fit_merge():
    # Read the first and second column from the file and plot them
    datal = np.loadtxt(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\AutoC_plot\fit_parameters_autocorrelationl.txt', delimiter='\t', skiprows=1)
    datam = np.loadtxt(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\AutoC_plot\fit_parameters_autocorrelationm.txt', delimiter='\t', skiprows=1)
    names = datal[:, 0]
    names = [f"{int(angle)}°"  for angle in datal[:, 0] ]

    fit_paramsl = datal[:, 1]
    fit_paramsm = datam[:, 1]
    # add legend

    plt.figure(figsize=(10, 5))
    plt.plot(names, fit_paramsl, color='red', marker='s', linestyle='--', linewidth=3, markersize=12, label='$\\phi$ = 0.00785')
    plt.plot(names, fit_paramsm, color='blue', marker='o', linestyle='--', linewidth=3, markersize=15, label='$\\phi$ = 0.0785')
    plt.tick_params(axis='both', which='major', direction='in', length=8, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.tick_params(axis='both', which='minor', direction='in', length=4, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
    plt.minorticks_on()
    plt.xticks(names[::2], rotation=45)  # Show only alternative labels on the x-axis
    plt.xlabel('$\\theta$', fontsize=20)
    plt.ylabel('$S$', fontsize=20)
    # add legend
    plt.legend(['$\\phi$ = 0.00785', '$\\phi$ = 0.0785'], loc='upper right', fontsize=15)
    # plt.show()
    plt.savefig(r'D:\Codebase\BasicFluidSimulation\RMC\Main\averaged_configuration\AutoC_plot\Fit_Parameter_metric_vs_Vision_Cone.png', dpi=600, bbox_inches='tight')
plot_auto_correlation_fit_merge()

# avang_cluster_plot

In [ ]:
packing_fraction=0.00785
# Function to read the data from the file
def read_data(filename):
    vision_angle = []
    angular_momentum = []
    cluster_size = []

    with open(filename, 'r') as file:
        for line in file:
            parts = line.split()
            if len(parts) >= 5:
                vision_angle.append(float(parts[1]))
                cluster_size.append(float(parts[2]))
                angular_momentum.append(float(parts[3]))
    return vision_angle, cluster_size, angular_momentum

# Read the data from the file
filename = r'RMC\Main\averaged_configuration\avdatalow.txt'
vision_angle, cluster_size, angular_momentum = read_data(filename)

fig, ax1 = plt.subplots(figsize=(10, 7))
vision_angle = [f"{int(angle)}°"  for angle in vision_angle ]

# Plotting the average angular momentum
ax1.plot(vision_angle, angular_momentum, 'b--o', label='Avg Angular Momentum', linewidth=2,markersize=10)
ax1.set_xlabel('$ \\theta$', fontsize=20)
ax1.set_ylabel('$\\langle L \\rangle$', color='b', fontsize=20)
ax1.tick_params(axis='y', labelcolor='b', labelsize=20)
ax1.tick_params(axis='x', labelsize=20)
# Create another y-axis for average cluster size
ax2 = ax1.twinx()
ax2.plot(vision_angle, cluster_size, 'r--s', label='Avg Cluster Size', linewidth=2,markersize=10)
ax2.set_ylabel('$\\langle C \\rangle$', color='r', fontsize=20)
ax2.tick_params(axis='y', labelcolor='r', labelsize=20)
plt.xticks(vision_angle[::2])  # Show only alternative labels on the x-axis
plt.tick_params(axis='both', which='major', direction='in', length=8, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
plt.tick_params(axis='both', which='minor', direction='in', length=4, width=2, bottom=True, top=True, left=True, right=True, labelsize=20)
plt.minorticks_on()
plt.title(f'$\\phi$ = {packing_fraction}', fontsize=20)
plt.savefig(r"RMC\Main\averaged_configuration\low.jpg")
